# Functions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.ticker import MaxNLocator
import math
from scipy import signal
from scipy.fftpack import fft
from scipy import ndimage
from scipy.io import loadmat
import os
%matplotlib inline

In [ ]:
import import_ipynb
os.chdir('C:\\Users\\hydro_leo\\Documents\\GitHub\\python-code\\Modules')
from time_shifted_PID import Histo3D2Dict
from time_shifted_PID import PIfunc

# Main

In [5]:
figmap = [[np.nan, 7,15,23,31,39,47,np.nan],
          [     1, 8,16,24,32,40,48,    55],
          [     2, 9,17,25,33,41,49,    56],
          [     3,10,18,26,34,42,50,    57],
          [     4,11,19,27,35,43,51,    58],
          [     5,12,20,28,36,44,52,    59],
          [     6,13,21,29,37,45,53,    60],
          [np.nan,14,22,30,38,46,54,np.nan]]
figmap = np.array(figmap)

In [6]:
import warnings
warnings.simplefilter('ignore')

for Fc in [0.5,1,2,4,8,0]:
    exp_folder = 'D:\\GoogleDrive\\retina\\Exps\\2020\\0729'
    filename = '0727_OUsmooth_Bright_UL_DR_G4.5_5min_Q100_6.5mW_'+str(Fc)+'Hz.mat'
    annots = loadmat(exp_folder+'\\merge\\'+'merge_'+filename)
    x = np.squeeze(annots['bin_pos'])
    x = x.astype(float)
    x = (x-np.mean(x))/np.std(x)
    T = np.squeeze(annots['diode_BT'])
    dt = 1./60.
    v = ndimage.gaussian_filter1d(x.copy(), sigma=5, order=1, mode='reflect')/dt

    xstate, _= EqualState(x, 6)
    vstate, _= EqualState(v, 6)
    PIfig, PIax = plt.subplots(nrows=8, ncols=8)
    MIfig, MIax = plt.subplots(nrows=8, ncols=8)
    for channelnumber in np.arange(1,61):
        rstate, _ = np.histogram(annots['reconstruct_spikes'][0][channelnumber-1], np.append(0,T))
        if sum(rstate)/len(rstate)/dt <= 0.3 : continue

        new_rstate = np.zeros_like(rstate, dtype = float)
        Spike = np.hstack((0, np.squeeze(annots['reconstruct_spikes'][0][channelnumber-1]), T[-1]))
        counter = 0
        for i in range(len(rstate)):
            counter += rstate[i]
            ISI = (Spike[counter+1]-Spike[counter-rstate[i]])/(rstate[i]+1)
            new_rstate[i] = ISI
        
        new_rstate, _= EqualState(new_rstate, 6)
        window = [-1,1] # second
        timeshift, Information = PIfunc(new_rstate, xstate, vstate, dt, window)
        if max(Information[('Beer','UIx')]) < 0.05: continue
        PIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIx')],'r-')
        PIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIv')],'b-')
        PIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','Syn')],'k-')
        PIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','Red')],'g-')
        PIax[np.where(figmap == channelnumber)][0].set_title(str(channelnumber))
        PIax[np.where(figmap == channelnumber)][0].grid()

        MIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIx')]+Information[('Beer','Red')],'r-')
        MIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIv')]+Information[('Beer','Red')],'b-')
        MIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIv')]+Information[('Beer','UIx')]+2*Information[('Beer','Red')],'m-')
        MIax[np.where(figmap == channelnumber)][0].plot(timeshift,Information[('Beer','UIv')]+Information[('Beer','UIx')]+Information[('Beer','Red')]+Information[('Beer','Syn')],'k-')
        MIax[np.where(figmap == channelnumber)][0].set_title(str(channelnumber))
        MIax[np.where(figmap == channelnumber)][0].grid()
    PIfig.set_size_inches(60, 30)
    PIfig.savefig(exp_folder+'\\MI\\TimeCodeFig//PI-'+filename+'.tif', bbox_inches = 'tight',pad_inches = 0.3)
    plt.close(PIfig)
    MIfig.set_size_inches(60, 30)
    MIfig.savefig(exp_folder+'\\MI\\TimeCodeFig//MI-'+filename+'.tif', bbox_inches = 'tight',pad_inches = 0.3)
    plt.close(MIfig)
    np.savez(exp_folder+'\\MI\\MI(timecode)-'+filename+'.npz', Information = Information, timeshift = timeshift)